# Image Segmentation

Trash Panda Data Science: Research and Testing

---

# Detectron2

*Similar to Mask R-CNN*

Resources:

- [How to embed Detectron2 in your cv project](https://medium.com/deepvisionguru/how-to-embed-detectron2-in-your-computer-vision-project-817f29149461)

## Install detectron2 and dependencies

In [1]:
# install dependencies:
# (use +cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 24kB/s 
     |████████████████████████████████| 4.1MB 106kB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 3.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=faadb36bac13220d70a838b7b806e1dc7e0084181ca5a483234f448604b9382e
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-47ahl_7e
  Running command git c

In [2]:
# install detectron2:
# !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Obtaining file:///content/detectron2_repo
  Running setup.py develop for detectron2


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

## Clone detectron2-pipeline

In [1]:
!git clone git://github.com/jagin/detectron2-pipeline.git

Cloning into 'detectron2-pipeline'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 117 (delta 37), reused 100 (delta 21), pack-reused 0
Receiving objects: 100% (117/117), 9.91 MiB | 30.93 MiB/s, done.
Resolving deltas: 100% (37/37), done.
fatal: not a git repository (or any of the parent directories): .git


In [5]:
%cd detectron2-pipeline

/content/detectron2-pipeline


In [6]:
!pwd

/content/detectron2-pipeline


In [7]:
!git checkout 9460e3806c3ef5208ba8e5b4099fcb75ef6f39d1

Note: checking out '9460e3806c3ef5208ba8e5b4099fcb75ef6f39d1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 9460e38 README.md update


---

## Background separation

In [9]:
%cd detectron2-pipeline/

[Errno 2] No such file or directory: 'detectron2-pipeline/'
/content/detectron2-pipeline


In [6]:
!python process_images.py -i assets/images/others/couple.jpg -sb

Traceback (most recent call last):
  File "process_images.py", line 7, in <module>
    from pipeline.predict import Predict
  File "/content/detectron2-pipeline/pipeline/predict.py", line 3, in <module>
    from detectron2.engine.defaults import DefaultPredictor
ModuleNotFoundError: No module named 'detectron2'


In [0]:
# pipeline/separate_background.py

from pipeline.pipeline import Pipeline

import numpy as np
import cv2


class SeparateBackground(Pipeline):

    def __init__(self, dst, me_kernel=(7, 7), bg_kernel=(21, 21), desaturate=True):
        self.dst = dst
        self.me_kernel = me_kernel  # mask edges gaussian blur kernel
        self.bg_kernel = bg_kernel  # background gaussian blur kernel
        self.desaturate = desaturate  # convert background to grayscale

        super().__init__()

    def map(self, data):
        self.separate_background(data)

        return data

    def separate_background(self, data):
        if "predictions" not in data:
            return

        predictions = data["predictions"]
        if "instances" not in predictions:
            return

        instances = predictions["instances"]
        if not instances.has("pred_masks"):
            return

        # Sum up all the instance masks
        mask = instances.pred_masks.cpu().sum(0) >= 1
        mask = mask.numpy().astype("uint8")*255
        # Create 3-channels mask
        mask = np.stack([mask, mask, mask], axis=2)

        # Apply a slight blur to the mask to soften edges
        mask = cv2.GaussianBlur(mask, self.me_kernel, 0)

        # Take the foreground input image
        foreground = data["image"]

        # Create a Gaussian blur for the background image
        background = cv2.GaussianBlur(foreground, self.bg_kernel, 0)

        if self.desaturate:
            # Convert background into grayscale
            background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

            # convert single channel grayscale image to 3-channel grayscale image
            background = cv2.cvtColor(background, cv2.COLOR_GRAY2RGB)

        # Convert uint8 to float
        foreground = foreground.astype(float)
        background = background.astype(float)

        # Normalize the alpha mask to keep intensity between 0 and 1
        mask = mask.astype(float)/255.0

        # Multiply the foreground with the mask
        foreground = cv2.multiply(foreground, mask)

        # Multiply the background with ( 1 - mask )
        background = cv2.multiply(background, 1.0 - mask)

        # Add the masked foreground and background
        dst_image = cv2.add(foreground, background)

        # Return a normalized output image for display
        data[self.dst] = dst_image.astype("uint8")

In [0]:
import os
from tqdm import tqdm
import multiprocessing as mp

from pipeline.capture_images import CaptureImages
from pipeline.capture_image import CaptureImage
from pipeline.predict import Predict
from pipeline.async_predict import AsyncPredict
from pipeline.separate_background import SeparateBackground
from pipeline.annotate_image import AnnotateImage
from pipeline.save_image import SaveImage
from pipeline.utils import detectron


def process_images(
    input_path,
    output_path = "output",
    config_file = "configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",
    weights_file = None,
    config_opts = [],
    confidence_threshold = 0.5,
    gpus = 1,
    cpus = 0,
    single_process = True,
    queue_size = 3,
    separate_background = True,
    progress = True,
):
    # Create output directory if needed
    os.makedirs(output_path, exist_ok=True)

    # Create pipeline steps
    capture_images = CaptureImages(input_path) \
        if os.path.isdir(input_path) else CaptureImage(input_path)

    cfg = detectron.setup_cfg(config_file=config_file,
                              weights_file=weights_file,
                              config_opts=config_opts,
                              confidence_threshold=confidence_threshold,
                              cpu=False if gpus > 0 else True)
    if not single_process:
        mp.set_start_method("spawn", force=True)
        predict = AsyncPredict(cfg,
                               num_gpus=gpus,
                               num_cpus=cpus,
                               queue_size=queue_size,
                               ordered=False)
    else:
        predict = Predict(cfg)

    if separate_background:
        separate_background = SeparateBackground("vis_image")
        annotate_image = None
    else:
        separate_background = None
        metadata_name = cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused"
        annotate_image = AnnotateImage("vis_image", metadata_name)

    save_image = SaveImage("vis_image", output_path)

    # Create image processing pipeline
    pipeline = (capture_images |
                predict |
                separate_background |
                annotate_image |
                save_image)

    # Iterate through pipeline
    try:
        for _ in tqdm(pipeline, disable=not progress):
            pass
    except StopIteration:
        return
    except KeyboardInterrupt:
        return
    finally:
        # Pipeline cleanup
        if isinstance(predict, AsyncPredict):
            predict.cleanup()

In [19]:
# Input the path as the only non-default parameter
input_path = "assets/images/others/couple.jpg"

process_images(input_path)


model_final_f10217.pkl: 178MB [00:06, 27.2MB/s]                           
1it [00:00,  3.03it/s]


In [20]:
# Test it out on some trashpanda training images
input_path = "assets/images/trashpanda"

process_images(input_path)


43it [00:07,  6.11it/s]
